In [1]:
#!pip install backtrader
#!pip install strategies
#!pip install yfinance
#!pip install matplotlib==3.2.2

In [2]:
import backtrader as bt
import matplotlib.pyplot as plt
import yfinance as yf

import argparse
import logging
import sys
from matplotlib import warnings

In [3]:
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] < self.dataclose[-1]:
                    # current close less than previous close

                    if self.dataclose[-1] < self.dataclose[-2]:
                        # previous close less than the previous close

                        # BUY, BUY, BUY!!! (with default parameters)
                        self.log('BUY CREATE, %.2f' % self.dataclose[0])

                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.buy()

        else:

            # Already in the market ... we might sell
            if len(self) >= (self.bar_executed + 5):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [4]:
class BuyAndHold_Buy(bt.Strategy):    
    def start(self):
        # set the starting cash
        self.val_start = self.broker.get_cash()     
    def nextstart(self):
        # Buy stocks with all the available cash
        size = int(self.val_start / self.data)
        self.buy(size=size)    
    def stop(self):
        # calculate the actual returns
        self.roi = (self.broker.get_value() / self.val_start) - 1.0
        print("ROI: %.2f, Cash: %.2f" % (100.0 * self.roi, self.broker.get_value()))

In [5]:
class BitmexComissionInfo(bt.CommissionInfo):
    params = (
        ("commission", 0.00075),
        ("mult", 1.0),
        ("margin", None),
        ("commtype", None),
        ("stocklike", False),
        ("percabs", False),
        ("interest", 0.0),
        ("interest_long", False),
        ("leverage", 1.0),
        ("automargin", False),
    )
def getsize(self, price, cash):
        """Returns fractional size for cash operation @price"""
        return self.p.leverage * (cash / price)

In [6]:
class MACD(bt.Strategy):
    params = (
        ("macd1", 12),
        ("macd2", 26),
        ("macdsig", 9),
        # Percentage of portfolio for a trade. Something is left for the fees
        # otherwise orders would be rejected
        ("portfolio_frac", 0.98),
    )

    def __init__(self):
        self.val_start = self.broker.get_cash()  # keep the starting cash
        self.size = None
        self.order = None

        self.macd = bt.ind.MACD(
            self.data,
            period_me1=self.p.macd1,
            period_me2=self.p.macd2,
            period_signal=self.p.macdsig,
        )
        # Cross of macd and macd signal
        self.mcross = bt.ind.CrossOver(self.macd.macd, self.macd.signal)
    
    def next(self):
        if self.order:
            return  # pending order execution. Waiting in orderbook

        print(
            f"DateTime {self.datas[0].datetime.datetime(0)}, "
            f"Price {self.data[0]:.2f}, Mcross {self.mcross[0]}, "
            f"Position {self.position.upopened}"
        )

        if not self.position:  # not in the market
            if self.mcross[0] > 0.0:
                print("Starting buy order")
                self.size = (
                    self.broker.get_cash() / self.datas[0].close * self.p.portfolio_frac
                )
                self.order = self.buy(size=self.size)
        else:  # in the market
            if self.mcross[0] < 0.0:
                print("Starting sell order")
                self.order = self.sell(size=self.size)
                
    def notify_order(self, order):
        """Execute when buy or sell is triggered
        Notify if order was accepted or rejected
        """
        if order.alive():
            print("Order is alive")
            # submitted, accepted, partial, created
            # Returns if the order is in a status in which it can still be executed
            return

        order_side = "Buy" if order.isbuy() else "Sell"
        if order.status == order.Completed:
            print(
                (
                    f"{order_side} Order Completed -  Size: {order.executed.size} "
                    f"@Price: {order.executed.price} "
                    f"Value: {order.executed.value:.2f} "
                    f"Comm: {order.executed.comm:.6f} "
                )
            )
        elif order.status in {order.Canceled, order.Margin, order.Rejected}:
            print(f"{order_side} Order Canceled/Margin/Rejected")
        self.order = None  # indicate no order pending

    def notify_trade(self, trade):
        """Execute after each trade
        Calcuate Gross and Net Profit/loss"""
        if not trade.isclosed:
            return
        print(f"Operational profit, Gross: {trade.pnl:.2f}, Net: {trade.pnlcomm:.2f}")

    def stop(self):
        """ Calculate the actual returns """
        self.roi = (self.broker.get_value() / self.val_start) - 1.0
        val_end = self.broker.get_value()
        print(
            f"ROI: {100.0 * self.roi:.2f}%%, Start cash {self.val_start:.2f}, "
            f"End cash: {val_end:.2f}"
        )

In [11]:
cerebro = bt.Cerebro()

cerebro.broker.set_cash(100000)

data = bt.feeds.YahooFinanceCSVData(dataname='Data/file.csv',reverse=False)

cerebro.resampledata(data, timeframe=bt.TimeFrame.Minutes, compression=60)

cerebro.addstrategy(MACD)

cerebro.broker.addcommissioninfo(BitmexComissionInfo())

#cerebro.broker.setcommission(commission=0.001)

# Add TimeReturn Analyzers to benchmark data
cerebro.addanalyzer(
    bt.analyzers.TimeReturn, _name="alltime_roi", timeframe=bt.TimeFrame.NoTimeFrame
)

cerebro.addanalyzer(
    bt.analyzers.TimeReturn,
    data=data,
    _name="benchmark",
    timeframe=bt.TimeFrame.NoTimeFrame,
)

results = cerebro.run()
st0 = results[0]

for alyzer in st0.analyzers:
    alyzer.print()

DateTime 2014-10-21 23:59:59.999989, Price 386.48, Mcross 0.0, Position 0
DateTime 2014-10-22 23:59:59.999989, Price 383.16, Mcross 0.0, Position 0
DateTime 2014-10-23 23:59:59.999989, Price 358.42, Mcross 0.0, Position 0
DateTime 2014-10-24 23:59:59.999989, Price 358.35, Mcross 0.0, Position 0
DateTime 2014-10-25 23:59:59.999989, Price 347.27, Mcross -1.0, Position 0
DateTime 2014-10-26 23:59:59.999989, Price 354.70, Mcross 0.0, Position 0
DateTime 2014-10-27 23:59:59.999989, Price 352.99, Mcross 0.0, Position 0
DateTime 2014-10-28 23:59:59.999989, Price 357.62, Mcross 0.0, Position 0
DateTime 2014-10-29 23:59:59.999989, Price 335.59, Mcross 0.0, Position 0
DateTime 2014-10-30 23:59:59.999989, Price 345.30, Mcross 0.0, Position 0
DateTime 2014-10-31 23:59:59.999989, Price 338.32, Mcross 0.0, Position 0
DateTime 2014-11-01 23:59:59.999989, Price 325.75, Mcross 0.0, Position 0
DateTime 2014-11-02 23:59:59.999989, Price 325.89, Mcross 0.0, Position 0
DateTime 2014-11-03 23:59:59.999989, 

DateTime 2015-10-08 23:59:59.999989, Price 242.30, Mcross 0.0, Position 481.9964354366065
DateTime 2015-10-09 23:59:59.999989, Price 243.93, Mcross 0.0, Position 481.9964354366065
DateTime 2015-10-10 23:59:59.999989, Price 244.94, Mcross 0.0, Position 481.9964354366065
DateTime 2015-10-11 23:59:59.999989, Price 247.05, Mcross 0.0, Position 481.9964354366065
DateTime 2015-10-12 23:59:59.999989, Price 245.31, Mcross 0.0, Position 481.9964354366065
DateTime 2015-10-13 23:59:59.999989, Price 249.51, Mcross 0.0, Position 481.9964354366065
DateTime 2015-10-14 23:59:59.999989, Price 251.99, Mcross 0.0, Position 481.9964354366065
DateTime 2015-10-15 23:59:59.999989, Price 254.32, Mcross 0.0, Position 481.9964354366065
DateTime 2015-10-16 23:59:59.999989, Price 262.87, Mcross 0.0, Position 481.9964354366065
DateTime 2015-10-17 23:59:59.999989, Price 270.64, Mcross 0.0, Position 481.9964354366065
DateTime 2015-10-18 23:59:59.999989, Price 261.64, Mcross 0.0, Position 481.9964354366065
DateTime 2

DateTime 2016-10-06 23:59:59.999989, Price 613.02, Mcross 0.0, Position 479.8121394756534
DateTime 2016-10-07 23:59:59.999989, Price 617.12, Mcross 0.0, Position 479.8121394756534
DateTime 2016-10-08 23:59:59.999989, Price 619.11, Mcross 0.0, Position 479.8121394756534
DateTime 2016-10-09 23:59:59.999989, Price 616.75, Mcross 0.0, Position 479.8121394756534
DateTime 2016-10-10 23:59:59.999989, Price 618.99, Mcross 0.0, Position 479.8121394756534
DateTime 2016-10-11 23:59:59.999989, Price 641.07, Mcross 0.0, Position 479.8121394756534
DateTime 2016-10-12 23:59:59.999989, Price 636.19, Mcross 0.0, Position 479.8121394756534
DateTime 2016-10-13 23:59:59.999989, Price 636.79, Mcross 0.0, Position 479.8121394756534
DateTime 2016-10-14 23:59:59.999989, Price 640.38, Mcross 0.0, Position 479.8121394756534
DateTime 2016-10-15 23:59:59.999989, Price 638.65, Mcross 0.0, Position 479.8121394756534
DateTime 2016-10-16 23:59:59.999989, Price 641.63, Mcross 0.0, Position 479.8121394756534
DateTime 2

DateTime 2017-09-29 23:59:59.999989, Price 4163.07, Mcross 0.0, Position 313.49674854091313
DateTime 2017-09-30 23:59:59.999989, Price 4338.71, Mcross 0.0, Position 313.49674854091313
DateTime 2017-10-01 23:59:59.999989, Price 4403.74, Mcross 0.0, Position 313.49674854091313
DateTime 2017-10-02 23:59:59.999989, Price 4409.32, Mcross 0.0, Position 313.49674854091313
DateTime 2017-10-03 23:59:59.999989, Price 4317.48, Mcross 0.0, Position 313.49674854091313
DateTime 2017-10-04 23:59:59.999989, Price 4229.36, Mcross 0.0, Position 313.49674854091313
DateTime 2017-10-05 23:59:59.999989, Price 4328.41, Mcross 0.0, Position 313.49674854091313
DateTime 2017-10-06 23:59:59.999989, Price 4370.81, Mcross 0.0, Position 313.49674854091313
DateTime 2017-10-07 23:59:59.999989, Price 4426.89, Mcross 0.0, Position 313.49674854091313
DateTime 2017-10-08 23:59:59.999989, Price 4610.48, Mcross 0.0, Position 313.49674854091313
DateTime 2017-10-09 23:59:59.999989, Price 4772.02, Mcross 0.0, Position 313.496

DateTime 2018-09-23 23:59:59.999989, Price 6710.63, Mcross 0.0, Position 525.3580230688136
DateTime 2018-09-24 23:59:59.999989, Price 6595.41, Mcross 0.0, Position 525.3580230688136
DateTime 2018-09-25 23:59:59.999989, Price 6446.47, Mcross 0.0, Position 525.3580230688136
DateTime 2018-09-26 23:59:59.999989, Price 6495.00, Mcross 0.0, Position 525.3580230688136
DateTime 2018-09-27 23:59:59.999989, Price 6676.75, Mcross 0.0, Position 525.3580230688136
DateTime 2018-09-28 23:59:59.999989, Price 6644.13, Mcross 0.0, Position 525.3580230688136
DateTime 2018-09-29 23:59:59.999989, Price 6601.96, Mcross 0.0, Position 525.3580230688136
DateTime 2018-09-30 23:59:59.999989, Price 6625.56, Mcross 0.0, Position 525.3580230688136
DateTime 2018-10-01 23:59:59.999989, Price 6589.62, Mcross 0.0, Position 525.3580230688136
DateTime 2018-10-02 23:59:59.999989, Price 6556.10, Mcross 0.0, Position 525.3580230688136
DateTime 2018-10-03 23:59:59.999989, Price 6502.59, Mcross 0.0, Position 525.3580230688136

DateTime 2019-08-27 23:59:59.999989, Price 10185.50, Mcross 0.0, Position 0
DateTime 2019-08-28 23:59:59.999989, Price 9754.42, Mcross 0.0, Position 0
DateTime 2019-08-29 23:59:59.999989, Price 9510.20, Mcross 0.0, Position 0
DateTime 2019-08-30 23:59:59.999989, Price 9598.17, Mcross 0.0, Position 0
DateTime 2019-08-31 23:59:59.999989, Price 9630.66, Mcross 0.0, Position 0
DateTime 2019-09-01 23:59:59.999989, Price 9757.97, Mcross 0.0, Position 0
DateTime 2019-09-02 23:59:59.999989, Price 10346.76, Mcross 0.0, Position 0
DateTime 2019-09-03 23:59:59.999989, Price 10623.54, Mcross 1.0, Position 0
Starting buy order
Order is alive
Order is alive
Buy Order Completed -  Size: 483.06407892847284 @Price: 10621.18 Value: 5130710.53 Comm: 38.480329 
DateTime 2019-09-04 23:59:59.999989, Price 10594.49, Mcross 0.0, Position 483.06407892847284
DateTime 2019-09-05 23:59:59.999989, Price 10575.53, Mcross 0.0, Position 483.06407892847284
DateTime 2019-09-06 23:59:59.999989, Price 10353.30, Mcross 0.

DateTime 2020-08-01 23:59:59.999989, Price 11759.59, Mcross 0.0, Position 669.4149701919938
DateTime 2020-08-02 23:59:59.999989, Price 11053.61, Mcross 0.0, Position 669.4149701919938
DateTime 2020-08-03 23:59:59.999989, Price 11246.35, Mcross 0.0, Position 669.4149701919938
DateTime 2020-08-04 23:59:59.999989, Price 11205.89, Mcross 0.0, Position 669.4149701919938
DateTime 2020-08-05 23:59:59.999989, Price 11747.02, Mcross 0.0, Position 669.4149701919938
DateTime 2020-08-06 23:59:59.999989, Price 11779.77, Mcross 0.0, Position 669.4149701919938
DateTime 2020-08-07 23:59:59.999989, Price 11601.47, Mcross 0.0, Position 669.4149701919938
DateTime 2020-08-08 23:59:59.999989, Price 11754.05, Mcross 0.0, Position 669.4149701919938
DateTime 2020-08-09 23:59:59.999989, Price 11675.74, Mcross 0.0, Position 669.4149701919938
DateTime 2020-08-10 23:59:59.999989, Price 11878.11, Mcross 0.0, Position 669.4149701919938
DateTime 2020-08-11 23:59:59.999989, Price 11410.53, Mcross -1.0, Position 669.4

DateTime 2021-08-07 23:59:59.999989, Price 44555.80, Mcross 0.0, Position 488.5107067650282
DateTime 2021-08-08 23:59:59.999989, Price 43798.12, Mcross 0.0, Position 488.5107067650282
DateTime 2021-08-09 23:59:59.999989, Price 46365.40, Mcross 0.0, Position 488.5107067650282
DateTime 2021-08-10 23:59:59.999989, Price 45585.03, Mcross 0.0, Position 488.5107067650282
DateTime 2021-08-11 23:59:59.999989, Price 45593.64, Mcross 0.0, Position 488.5107067650282
DateTime 2021-08-12 23:59:59.999989, Price 44428.29, Mcross 0.0, Position 488.5107067650282
DateTime 2021-08-13 23:59:59.999989, Price 47793.32, Mcross 0.0, Position 488.5107067650282
DateTime 2021-08-14 23:59:59.999989, Price 47096.95, Mcross 0.0, Position 488.5107067650282
DateTime 2021-08-15 23:59:59.999989, Price 47047.00, Mcross 0.0, Position 488.5107067650282
DateTime 2021-08-16 23:59:59.999989, Price 46004.48, Mcross 0.0, Position 488.5107067650282
DateTime 2021-08-17 23:59:59.999989, Price 44695.36, Mcross -1.0, Position 488.5

In [12]:
cerebro.plot()

ImportError: cannot import name 'warnings' from 'matplotlib.dates' (C:\Users\Welcome\anaconda3\lib\site-packages\matplotlib\dates.py)

In [ ]:
'''
data = bt.feeds.YahooFinanceCSVData(dataname='Data/file.csv',reverse=False)

cerebro = bt.Cerebro()

cerebro.adddata(data)

cerebro.addstrategy(TestStrategy)

#cerebro.addstrategy(BuyAndHold_Buy)

cerebro.broker.addcommissioninfo(BitmexComissionInfo())

cerebro.broker.setcommission(commission=0.001)

cerebro.broker.setcash(10000.0)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
'''